## COPERNICUS Vegetation Indicatorss 

### <a name="service">Service Definition

In [1]:
service = dict([('title', 'COPERNICUS Vegetation Indicators - Aggregations'),
                ('abstract', 'COPERNICUS Vegetation Indicators - Aggregations'),
                ('id', 'wfp-01-03-01')])

### <a name="parameter">Parameter Definition 

In [2]:
N_1 = dict([('id', 'N_1'),
                          ('value', 'True'),
                          ('title', 'No Aggregation'),
                          ('abstract', 'No aggregation')])

In [3]:
N_3 = dict([('id', 'N_3'),
                          ('value', 'True'),
                          ('title', '30 Day Aggregation'),
                          ('abstract', 'Get a 30 day aggregation')])

In [4]:
N_6 = dict([('id', 'N_6'),
                          ('value', 'False'),
                          ('title', '60 Day Aggregation'),
                          ('abstract', 'Get a 60 day aggregation')])

In [5]:
N_9 = dict([('id', 'N_9'),
                          ('value', 'False'),
                          ('title', '90 Day Aggregation'),
                          ('abstract', 'Get a 90 day aggregation')])

In [6]:
N_12 = dict([('id', 'N_12'),
                          ('value', 'False'),
                          ('title', '120 Day Aggregation'),
                          ('abstract', 'Get a 120 day aggregation')])

In [7]:
N_15 = dict([('id', 'N_15'),
                          ('value', 'False'),
                          ('title', '150 Day Aggregation'),
                          ('abstract', 'Get a 150 day aggregation')])

In [8]:
N_18 = dict([('id', 'N_18'),
                          ('value', 'False'),
                          ('title', '180 Day Aggregation'),
                          ('abstract', 'Get a 180 day aggregation')])

In [9]:
N_27 = dict([('id', 'N_27'),
                          ('value', 'False'),
                          ('title', '270 Day Aggregation'),
                          ('abstract', 'Get a 270 day aggregation')])

In [10]:
N_36 = dict([('id', 'N_36'),
                          ('value', 'False'),
                          ('title', '360 Day Aggregation'),
                          ('abstract', 'Get a 360 day aggregation')])

In [11]:
regionOfInterest = dict([('id', 'regionOfInterest'),
                          ('value', 'POLYGON((11.5030755518998 -11.1141633706909,41.0343255518998 -11.1141633706909,41.0343255518998 -34.9763656693858,11.5030755518998 -34.9763656693858,11.5030755518998 -11.1141633706909))'),
                          ('title', 'WKT Polygon for the Region of Interest'),
                          ('abstract', 'Set the value of WKT Polygon')])

In [12]:
nameOfRegion = dict([('id', 'nameOfRegion'),
                     ('value', ''),
                     ('title', 'Name of Region'),
                     ('abstract', 'Name of the region of interest'),
                     ('minOccurs', '1')])

In [13]:
lta_url = dict([('id', 'lta_url'),
                ('value', 'https://catalog.terradue.com//better-wfp-00009/series/results/search'),
                ('title', 'Catalogue Url for the LTA products'),
                ('abstract', 'Catalogue Url for anomalies')])

### <a name="runtime">Runtime parameter definition

**Input references**

In [14]:
input_references = ('https://catalog.terradue.com/cgls/search?format=atom&uid=fapar_v2_1km_FAPAR-RT6_201702280000_GLOBE_PROBAV_V2.0.2')

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [15]:
import os
import sys
#sys.path.append('/opt/OTB-6.2.0/lib/python')
#sys.path.append('/opt/OTB-6.2.0/lib/libfftw3.so.3')
#os.environ['OTB_APPLICATION_PATH'] = '/opt/OTB-6.2.0/lib/otb/applications'
#os.environ['LD_LIBRARY_PATH'] = '/opt/OTB-6.2.0/lib'
#os.environ['ITK_AUTOLOAD_PATH'] = '/opt/OTB-6.2.0/lib/otb/applications'
#os.environ['GDAL_DATA'] = '/opt/anaconda/share/gdal/'
#import otbApplication
from osgeo import gdal
import numpy as np
import pandas as pd
from geopandas import GeoDataFrame
from datetime import datetime
import math
import re
import cioppy
import shutil
import requests
from shapely.wkt import loads
sys.path.append('/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec')
sys.path.append('/application/notebook/libexec')
from aux_functions import calc_average, calc_max_matrix, matrix_sum, crop_image, write_output_image, get_matrix_list

In [16]:
def get_info(row, search_params):
    search = ciop.search(end_point=row['catalogue_url'], 
                                  params=search_params,
                                  output_fields='self,identifier,startdate,enclosure',
                                  model='GeoTime')[0]
    
    series = pd.Series(search)
    
    series['startdate'] = pd.to_datetime(series['startdate'])
    
    return series

In [17]:
def get_lta_list(url, search_params):
    search = ciop.search(end_point=url,
                         params=search_params,
                         output_fields='self,identifier,enclosure,title,startdate,enddate,wkt,updated',
                         model='GeoTime')
    return search

In [18]:
if isinstance(input_references, str):
    input_references = [input_references]


region_of_interest = regionOfInterest['value']
nameOfRegion = nameOfRegion['value']
#nameOfRegion = 'SouthernAfrica'
ciop = cioppy.Cioppy()

lai_references = [ref for ref in input_references if 'LAI' in ref]
fapar_references = [ref for ref in input_references if 'FAPAR' in ref]
gpd_lai_final = None
gpd_fapar_final = None
if lai_references:
    gpd_data = GeoDataFrame(lai_references,
                           columns=['catalogue_url'])

    gpd_data = gpd_data.sort_values(by='catalogue_url')
    print(gpd_data['catalogue_url'])
    start_date = re.findall('\d{8}0000', gpd_data.iloc[0]['catalogue_url'])[0].replace('0000', '')
    end_date = re.findall('\d{8}0000', gpd_data.iloc[-1]['catalogue_url'])[0].replace('0000', '')
    start_date = re.sub(r'(\d{4})(\d{2})(\d{2})', r'\1-\2-\3', start_date)
    end_date = re.sub(r'(\d{4})(\d{2})(\d{2})', r'\1-\2-\3', end_date)
    print(start_date)
    print(end_date)
    print(len(lai_references))
    search_params =  dict([('start', start_date),
                          ('stop', end_date),
                          ('count', len(lai_references))])

    gpd_lai_final = gpd_data.apply(lambda row: get_info(row, search_params), axis=1)

if fapar_references:
    gpd_data = GeoDataFrame(fapar_references,
                           columns=['catalogue_url'])

    gpd_data = gpd_data.sort_values(by='catalogue_url')
    print(gpd_data['catalogue_url'])
    start_date = re.findall('\d{8}0000', gpd_data.iloc[0]['catalogue_url'])[0].replace('0000', '')
    end_date = re.findall('\d{8}0000', gpd_data.iloc[-1]['catalogue_url'])[0].replace('0000', '')
    start_date = re.sub(r'(\d{4})(\d{2})(\d{2})', r'\1-\2-\3', start_date)
    end_date = re.sub(r'(\d{4})(\d{2})(\d{2})', r'\1-\2-\3', end_date)
    print(start_date)
    print(end_date)
    print(len(fapar_references))
    search_params =  dict([('start', start_date),
                          ('stop', end_date),
                          ('count', len(fapar_references))])

    gpd_fapar_final = gpd_data.apply(lambda row: get_info(row, search_params), axis=1)

2    https://catalog.terradue.com/cgls/search?forma...
1    https://catalog.terradue.com/cgls/search?forma...
0    https://catalog.terradue.com/cgls/search?forma...
5    https://catalog.terradue.com/cgls/search?forma...
4    https://catalog.terradue.com/cgls/search?forma...
3    https://catalog.terradue.com/cgls/search?forma...
Name: catalogue_url, dtype: object
2017-01-10
2017-02-28
6
5    https://catalog.terradue.com/cgls/search?forma...
4    https://catalog.terradue.com/cgls/search?forma...
3    https://catalog.terradue.com/cgls/search?forma...
2    https://catalog.terradue.com/cgls/search?forma...
1    https://catalog.terradue.com/cgls/search?forma...
0    https://catalog.terradue.com/cgls/search?forma...
Name: catalogue_url, dtype: object
2017-01-10
2017-02-28
6


In [19]:
lta_params = dict([('start', start_date + 'T00:00:00.0000000Z'),
                   ('stop', end_date + 'T00:00:00.0000000Z'),
                   ('geom', region_of_interest),
                   ('count', 150)])
print(lta_params)
lta_data = get_lta_list(lta_url['value'], lta_params)
#lta_data = GeoDataFrame(lta_list, columns=['catalogue_url'])
#lta_data = lta_data.apply(lambda row: get_lta_info(row,[]), axis=1)

lta_data = GeoDataFrame.from_dict(lta_data)
lta_data['startdate'] = pd.to_datetime(lta_data['startdate'])
lta_data['enddate'] = pd.to_datetime(lta_data['enddate'])
lta_data['wkt'] = lta_data['wkt'].apply(lambda row: loads(row))
lta_data.head(20)

{'count': 150, 'start': '2017-01-10T00:00:00.0000000Z', 'geom': 'POLYGON((11.5030755518998 -11.1141633706909,41.0343255518998 -11.1141633706909,41.0343255518998 -34.9763656693858,11.5030755518998 -34.9763656693858,11.5030755518998 -11.1141633706909))', 'stop': '2017-02-28T00:00:00.0000000Z'}


,enclosure,enddate,identifier,self,startdate,title,updated,wkt
0,https://store.terradue.com/better-wfp-00009/_r...,2017-01-31,01ED57858120D350BCD467224037F18C964F6CD5,https://catalog.terradue.com//better-wfp-00009...,2016-01-01,Output LTA_CHIRPSv2_BotswanaZambia_N30_daystot...,2019-03-11T15:39:44.3101830+00:00,"POLYGON ((19.5641183795274 -14.9105384014693, ..."
1,https://store.terradue.com/better-wfp-00009/_r...,2017-01-31,0C12098705F6C606B0A946D85A7A39770D5D461F,https://catalog.terradue.com//better-wfp-00009...,2015-01-01,Output LTA_CHIRPSv2_SouthernAfrica_N30_dryspel...,2019-03-11T15:39:44.3101810+00:00,"POLYGON ((11.5030755518998 -11.1141633706909, ..."
2,https://store.terradue.com/better-wfp-00009/_r...,2017-01-31,147931514ADF3BD9A3EB3C72F097224304197E8C,https://catalog.terradue.com//better-wfp-00009...,2015-01-10,Output LTA_LAI_SouthernAfrica_N3_averages_1-10...,2019-03-11T16:09:09.0879070+00:00,"POLYGON ((11.5030755518998 -11.1141633706909, ..."
3,https://store.terradue.com/better-wfp-00009/_r...,2017-01-31,55813C2ADE9BD1824A03AF21CE7740A22A8CD0BF,https://catalog.terradue.com//better-wfp-00009...,2015-01-10,Output LTA_FAPAR_SouthernAfrica_N3_averages_1-...,2019-03-11T16:09:09.0879080+00:00,"POLYGON ((11.5030755518998 -11.1141633706909, ..."
4,https://store.terradue.com/better-wfp-00009/_r...,2017-01-31,6AD86AB8988A776CB2939FBEF5AD36B52050B099,https://catalog.terradue.com//better-wfp-00009...,2016-01-01,Output LTA_CHIRPSv2_BotswanaZambia_N30_countab...,2019-03-11T15:39:44.3101830+00:00,"POLYGON ((19.5641183795274 -14.9105384014693, ..."
5,https://store.terradue.com/better-wfp-00009/_r...,2017-01-31,7558809279D798348CD475AACAA52869D963A76E,https://catalog.terradue.com//better-wfp-00009...,2015-01-01,Output LTA_CHIRPSv2_SouthernAfrica_N30_daystot...,2019-03-11T15:39:44.3101810+00:00,"POLYGON ((11.5030755518998 -11.1141633706909, ..."
6,https://store.terradue.com/better-wfp-00009/_r...,2017-01-31,86A0BE56B5DFBDE03197DCE0E3491F36BDA769A2,https://catalog.terradue.com//better-wfp-00009...,2015-01-01,Output LTA_CHIRPSv2_SouthernAfrica_N30_countab...,2019-03-11T15:39:44.3101820+00:00,"POLYGON ((11.5030755518998 -11.1141633706909, ..."
7,https://store.terradue.com/better-wfp-00009/_r...,2017-01-31,9703BEB32893CE6719C905914CC52E349846F568,https://catalog.terradue.com//better-wfp-00009...,2016-01-01,Output LTA_CHIRPSv2_BotswanaZambia_N30_dryspel...,2019-03-11T15:39:44.3101820+00:00,"POLYGON ((19.5641183795274 -14.9105384014693, ..."
8,https://store.terradue.com/better-wfp-00009/_r...,2017-01-31,9BF5187EBF0CE31AEB91766552F8AB76F7C0B690,https://catalog.terradue.com//better-wfp-00009...,2015-01-10,Output LTA_LAI_SouthernAfrica_N3_maxvalues_1-1...,2019-03-11T16:09:09.0879070+00:00,"POLYGON ((11.5030755518998 -11.1141633706909, ..."
9,https://store.terradue.com/better-wfp-00009/_r...,2017-01-31,F4DE56825DB1A70F233AE41C21F453280DB0B8E9,https://catalog.terradue.com//better-wfp-00009...,2015-01-10,Output LTA_FAPAR_SouthernAfrica_N3_maxvalues_1...,2019-03-11T16:09:09.0879080+00:00,"POLYGON ((11.5030755518998 -11.1141633706909, ..."


In [20]:
def get_lta_from_dataframe(dataframe, region_name, product_type, aggregation, start_day, end_day, start_month, end_month, wkt):
    region_polygon = loads(wkt)
    dataframe_agr = dataframe[(dataframe['title'].str.contains(aggregation)) &
                              (dataframe['title'].str.contains(product_type)) &
                              (dataframe['title'].str.contains(region_name))]
    period_lta = dataframe_agr[(dataframe_agr['startdate'].dt.day == start_day) & 
                     (dataframe_agr['startdate'].dt.month == start_month) & 
                     (dataframe_agr['enddate'].dt.day == end_day) & 
                     (dataframe_agr['enddate'].dt.month == end_month) &
                     (dataframe_agr['wkt'] == region_polygon)]
    
    if len(period_lta.index.values) > 1:
        outdated_indexes = period_lta[period_lta['updated'] != max(period_lta['updated'])].index.values
        period_lta = period_lta.drop(outdated_indexes)
    return period_lta['enclosure'].tolist()


In [21]:
def get_product(url, dest):
    
    r = requests.get(url)
    
    open(dest, 'wb').write(r.content)
    
    return r.status_code

In [22]:
def calc_aggregations(product_list, N_value, region_of_interest, product_type):
    mask_no_data_value = 0
    max_values = 0
    averages = 0
    temp_list = []
    no_data_value = None
    print(type(product_list))
    for product_url in product_list:
        # uncompressed data
        product = (product_url.split('/')[-1]).split('.gz')[0]
        print(product)
        cropped_product_path = 'crop_' + product
        cropped_product_path = cropped_product_path.split('.nc')[0] + '.tif'
        try:
            crop_image(product_url, region_of_interest, cropped_product_path, product_type)
            # Read GeoTIFF as an array
            dataset = gdal.Open(cropped_product_path)
            product_array = dataset.GetRasterBand(1).ReadAsArray()
            no_data_value = dataset.GetRasterBand(1).GetNoDataValue()
            print(no_data_value)
            geo_transform = dataset.GetGeoTransform()
            projection = dataset.GetProjection()
            ## Create mask of no_data_values
            if no_data_value is not None:
                if isinstance(mask_no_data_value, int):
                    mask_no_data_value = np.where(product_array == no_data_value, 1, 0)
                else:
                    temp_mask = np.where(product_array == no_data_value, 1, 0)
                    mask_no_data_value = matrix_sum(mask_no_data_value, temp_mask)
            max_values = calc_max_matrix(max_values, product_array, no_data_value)
            temp_list.append(product_array)
            dataset = None
            
        except Exception as e:
            print('Error processing the product ' + product + ': ' + str(e))
        if os.path.exists(cropped_product_path):
            os.remove(cropped_product_path)
    
    averages = calc_average(temp_list, N_value, no_data_value)
    
    return max_values, averages, mask_no_data_value, geo_transform, projection

In [23]:
def product_stage_in(enclosure, target_dir):
    if not os.path.isdir(target_dir):
        os.mkdir(target_dir)
    return ciop.copy(urls=enclosure, target=target_dir)

In [24]:
def write_outputs(product_name, roi_name, first_date, last_date, averages, max_values, mask_no_data_value, image_format, product_count, projection, geo_transform, no_data_value):
    filenames = []
    filenames.append(product_name + '_' + roi_name + '_N' + str(product_count) + '_averages_' + first_date + '_' + last_date + '.tif')
    filenames.append(product_name + '_' + roi_name + '_N' + str(product_count) + '_maxvalues_' + first_date + '_' + last_date + '.tif')
    write_output_image(filenames[0], averages, image_format,  mask_no_data_value, projection, geo_transform, no_data_value)
    write_output_image(filenames[1], max_values, image_format, mask_no_data_value, projection, geo_transform, no_data_value)
    return filenames

In [25]:
def write_anomaly_output(anomaly, product_name, first_date, last_date, lta_start_year, lta_end_year, aggregation, mask_no_value, N_value, regionOfInterest, roi_name, projection, geo_transform, no_data_value):
    #image_number = (datetime.strptime(last_date, '%Y-%m-%d') - datetime.strptime(first_date, '%Y-%m-%d')).days
    filename =  product_name + 'Anomaly_' + roi_name + '_N' + str(N_value) + '_' + aggregation + '_' + first_date + '_' + last_date + '_LTA' + str(lta_start_year) + '_' + str(lta_end_year) + '.tif'
    write_output_image(filename, anomaly, 'GTiff', mask_no_value, projection, geo_transform, no_data_value)
    return filename

In [26]:
def get_formatted_date(product_reference):
    metadata = ciop.search(end_point=product_reference,
                           params=[],
                           output_fields='identifier,startdate',
                           model="GeoTime")
    return metadata[0]['startdate']

In [27]:
def write_properties_file(dataframe, output_name):
    
    title = 'Output %s' % output_name
    first_date = get_formatted_date(dataframe.iloc[0]['self'])
    last_date = get_formatted_date(dataframe.iloc[-1]['self'])
    with open(output_name + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (first_date, last_date))
        file.write('geometry=%s' % (regionOfInterest['value']))

In [28]:
nlist = [N_1['value'], N_3['value'], N_6['value'], N_9['value'], N_12['value'], N_15['value'], N_18['value'], N_27['value'], N_36['value']]
nvalue = [1, 3, 6, 9, 12, 15, 18, 27, 36]
#L = len(gpd_final.index.values)
nlist = [n=='True' for n in nlist]
print(nlist)
if gpd_lai_final is not None:
    lai_product_years = gpd_lai_final['startdate'].dt.year.unique()
else:
    lai_product_years = []
if gpd_fapar_final is not None:
    fapar_product_years = gpd_fapar_final['startdate'].dt.year.unique()
else:
    fapar_product_years = []
product_years = list(lai_product_years) + list(set(fapar_product_years) - set(lai_product_years))
print(product_years)
for _year in product_years:
    #products_data = gpd_final[(gpd_final['startdate'].dt.year == _year)]
    #lai_products = products_data[(products_data['identifier'].str.contains('lai'))]
    #fapar_products = products_data[(products_data['identifier'].str.contains('fapar'))]
    lai_products = gpd_lai_final[(gpd_lai_final['startdate'].dt.year == _year)]
    fapar_products = gpd_fapar_final[(gpd_fapar_final['startdate'].dt.year == _year)]
    if not lai_products.empty:
        products_data = lai_products
        L = len(products_data.index.values)
        months_of_products = products_data['startdate'].dt.month.unique()
        print(months_of_products)
        for n in [index for index, value in enumerate(nlist) if value==True]:
            N = nvalue[n]
            n_months = N/3
            if N == 1:
                pass
            else: 
                if N == 3:
                    for month_idx in months_of_products:
                        print(month_idx)
                        start_date = products_data[(products_data['startdate'].dt.month == month_idx)].iloc[0]['startdate']
                        end_date = products_data[(products_data['startdate'].dt.month == month_idx)].iloc[-1]['startdate']
                        interval_gpd = products_data[(products_data['startdate'] >= start_date) & (products_data['startdate'] <= end_date)]
                        interval_products = interval_gpd['enclosure'].tolist()
                        
                        file_list = []
                        for enclosure in interval_products:
                            filepath = product_stage_in(enclosure, 'tmp_data')
                            file_list.append(filepath)
                            print(file_list)
                        if file_list:
                            first_date = start_date.strftime('%Y-%m-%d')
                            last_date = end_date.strftime('%Y-%m-%d')
                            print(first_date)
                            print(last_date)
                            max_values, averages, no_value, geo_transform, projection = calc_aggregations(file_list, N, region_of_interest, 'LAI')
                        
                            
                            LTA_max_values = get_lta_from_dataframe(lta_data, 'LAI', 'maxvalues', nameOfRegion, start_date.day, end_date.day, start_date.month, end_date.month, region_of_interest)
                            LTA_averages = get_lta_from_dataframe(lta_data, 'LAI', 'averages', nameOfRegion, start_date.day, end_date.day, start_date.month, end_date.month, region_of_interest)
                            print(LTA_max_values)
                            print(LTA_averages)
                            
                            anomaly_files = []
                            print('Calculating anomalies...')
                            if len(LTA_max_values) > 0:
                                LTA_max_values = LTA_max_values[0]
                                start_year = re.findall('\d{4}', os.path.basename(LTA_max_values))[0]
                                end_year = re.findall('\d{4}', os.path.basename(LTA_max_values))[1]
                                print(start_year)
                                print(end_year)
                                filepath = 'tmp_data/' + os.path.basename(LTA_max_values)
                                status = get_product(LTA_max_values, filepath)
                                if status == 200:
                                    filepath = [filepath]
                                    LTA_max_values = get_matrix_list(filepath)[0]
                                    anomaly_max_values = np.divide(max_values, LTA_max_values)
                                    filename = write_anomaly_output(anomaly_max_values, 'LAI', first_date, last_date, start_year, end_year, 'maxvalues', no_value, N, region_of_interest, nameOfRegion, projection, geo_transform, -999.0)
                                    anomaly_files.append(filename)
                                    os.remove(filepath[0])
                            else:
                                period_str = str(start_date.month) + '-' + str(start_date.day) + '/' + str(end_date.month) + '-' + str(end_date.day)
                                message = 'No Long-term Average match found for "maxvalues" aggregation from the period (month-day): ' + period_str 
                                ciop.log('INFO', message)
                                
                            if len(LTA_averages) > 0:
                                LTA_averages = LTA_averages[0]
                                start_year = re.findall('\d{4}', os.path.basename(LTA_averages))[0]
                                end_year = re.findall('\d{4}', os.path.basename(LTA_averages))[1]
                                print(start_year)
                                print(end_year)
                                filepath = 'tmp_data/' + os.path.basename(LTA_averages)
                                status = get_product(LTA_averages, filepath)
                                if status == 200:
                                    filepath = [filepath]
                                    LTA_averages = get_matrix_list(filepath)[0]
                                    anomaly_averages = np.divide(averages, LTA_averages)
                                    filename = write_anomaly_output(anomaly_averages, 'LAI', first_date, last_date, start_year, end_year, 'averages', no_value, N, region_of_interest, nameOfRegion, projection, geo_transform, -999.0)
                                    anomaly_files.append(filename)
                                    os.remove(filepath[0])
                                
                            else:
                                period_str = str(start_date.month) + '-' + str(start_date.day) + '/' + str(end_date.month) + '-' + str(end_date.day)
                                message = 'No Long-term Average match found for "averages" aggregation from the period (month-day): ' + period_str 
                                ciop.log('INFO', message)
                            
                        
                            filenames = write_outputs('LAI', nameOfRegion, first_date, last_date, averages, max_values, no_value, 'GTiff', N, projection, geo_transform, -999.0)
                            filenames = filenames + anomaly_files
                            for output_name in filenames:
                                write_properties_file(interval_gpd, output_name)
                            for tmp_file in file_list:
                                os.remove(tmp_file)
                        else:
                            print('ERROR: No LAI product files found.')
                else:
                    for month_idx in range(months_of_products[0], months_of_products[-1], n_months):
                        start_date = products_data[(products_data['startdate'].dt.month == month_idx)].iloc[0]['startdate']
                        end_date = products_data[(products_data['startdate'].dt.month == month_idx + n_months-1)].iloc[-1]['startdate']
                        interval_gpd = products_data[(products_data['startdate'] >= start_date) & (products_data['startdate'] <= end_date)]
                        interval_products = interval_gpd['enclosure'].tolist()
                        
                        file_list = []
                        for enclosure in interval_products:
                            filepath = product_stage_in(enclosure, 'tmp_data')
                            file_list.append(filepath)
                            print(file_list)
                        if file_list:
                            first_date = start_date.strftime('%Y-%m-%d')
                            last_date = end_date.strftime('%Y-%m-%d')
                            print(start_date)
                            print(end_date)
                            max_values, averages, no_value, geo_transform, projection = calc_aggregations(file_list, N, region_of_interest, 'LAI')
                        
                            LTA_max_values = get_lta_from_dataframe(lta_data, 'LAI', 'maxvalues', nameOfRegion, start_date.day, end_date.day, start_date.month, end_date.month, region_of_interest)
                            LTA_averages = get_lta_from_dataframe(lta_data, 'LAI', 'averages', nameOfRegion, start_date.day, end_date.day, start_date.month, end_date.month, region_of_interest)
                            print(LTA_max_values)
                            print(LTA_averages)
                            
                            anomaly_files = []
                            print('Calculating anomalies...')
                            if len(LTA_max_values) > 0:
                                LTA_max_values = LTA_max_values[0]
                                start_year = re.findall('\d{4}', os.path.basename(LTA_max_values))[0]
                                end_year = re.findall('\d{4}', os.path.basename(LTA_max_values))[1]
                                print(start_year)
                                print(end_year)
                                filepath = 'tmp_data/' + os.path.basename(LTA_max_values)
                                status = get_product(LTA_max_values, filepath)
                                if status == 200:
                                    filepath = [filepath]
                                    LTA_max_values = get_matrix_list(filepath)[0]
                                    anomaly_max_values = np.divide(max_values, LTA_max_values)
                                    filename = write_anomaly_output(anomaly_max_values, 'LAI', first_date, last_date, start_year, end_year, 'maxvalues', no_value, N, region_of_interest, nameOfRegion, projection, geo_transform, -999.0)
                                    anomaly_files.append(filename)
                                    os.remove(filepath[0])
                            
                            else:
                                period_str = str(start_date.month) + '-' + str(start_date.day) + '/' + str(end_date.month) + '-' + str(end_date.day)
                                message = 'No Long-term Average match found for "maxvalues" aggregation from the period (month-day): ' + period_str 
                                ciop.log('INFO', message)
                            
                            if len(LTA_averages) > 0:
                                LTA_averages = LTA_averages[0]
                                start_year = re.findall('\d{4}', os.path.basename(LTA_averages))[0]
                                end_year = re.findall('\d{4}', os.path.basename(LTA_averages))[1]
                                filepath = 'tmp_data/' + os.path.basename(LTA_averages)
                                status = get_product(LTA_averages, filepath)
                                if status == 200:
                                    filepath = [filepath]
                                    LTA_averages = get_matrix_list(filepath)[0]
                                    
                                    anomaly_averages = np.divide(averages, LTA_averages)
                                    filename = write_anomaly_output(anomaly_averages, 'LAI', first_date, last_date, start_date.year, end_date.year, 'averages', no_value, N, region_of_interest, nameOfRegion, projection, geo_transform, -999.0)
                                    anomaly_files.append(filename)
                                    os.remove(filepath[0])
                            
                            else:
                                period_str = str(start_date.month) + '-' + str(start_date.day) + '/' + str(end_date.month) + '-' + str(end_date.day)
                                message = 'No Long-term Average match found for "averages" aggregation from the period (month-day): ' + period_str 
                                ciop.log('INFO', message)
                        
                            filenames = write_outputs('LAI', nameOfRegion, first_date, last_date, averages, max_values, no_value, 'GTiff', N, projection, geo_transform, -999.0)
                            filenames = filenames + anomaly_files
                            for output_name in filenames:
                                write_properties_file(interval_gpd, output_name)
                            for tmp_file in file_list:
                                os.remove(tmp_file)
                        else:
                            print('ERROR: No LAI product files found.')
    
    if not fapar_products.empty:
        products_data = fapar_products
        L = len(products_data.index.values)
        months_of_products = products_data['startdate'].dt.month.unique()
        print(months_of_products)
        for n in [index for index, value in enumerate(nlist) if value==True]:
            N = nvalue[n]
            n_months = N/3
            if N == 1:
                pass
            else: 
                if N == 3:
                    for month_idx in months_of_products:
                        print(month_idx)
                        start_date = products_data[(products_data['startdate'].dt.month == month_idx)].iloc[0]['startdate']
                        end_date = products_data[(products_data['startdate'].dt.month == month_idx)].iloc[-1]['startdate']
                        interval_gpd = products_data[(products_data['startdate'] >= start_date) & (products_data['startdate'] <= end_date)]
                        interval_products = interval_gpd['enclosure'].tolist()
                        
                        file_list = []
                        for enclosure in interval_products:
                            filepath = product_stage_in(enclosure, 'tmp_data')
                            file_list.append(filepath)
                            print(file_list)
                        if file_list:
                            first_date = start_date.strftime('%Y-%m-%d')
                            last_date = end_date.strftime('%Y-%m-%d')
                            print(start_date)
                            print(end_date)
                            max_values, averages, no_value, geo_transform, projection = calc_aggregations(file_list, N, region_of_interest, 'FAPAR')
                        
                            LTA_max_values = get_lta_from_dataframe(lta_data, 'FAPAR', 'maxvalues', nameOfRegion, start_date.day, end_date.day, start_date.month, end_date.month, region_of_interest)
                            LTA_averages = get_lta_from_dataframe(lta_data, 'FAPAR', 'averages', nameOfRegion, start_date.day, end_date.day, start_date.month, end_date.month, region_of_interest)
                            print(LTA_max_values)
                            print(LTA_averages)
                            
                            anomaly_files = []
                            print('Calculating anomalies...')
                            if len(LTA_max_values) > 0:
                                LTA_max_values = LTA_max_values[0]
                                start_year = re.findall('\d{4}', os.path.basename(LTA_max_values))[0]
                                end_year = re.findall('\d{4}', os.path.basename(LTA_max_values))[1]
                                print(start_year)
                                print(end_year)
                                filepath = 'tmp_data/' + os.path.basename(LTA_max_values)
                                status = get_product(LTA_max_values, filepath)
                                if status == 200:
                                    filepath = [filepath]
                                    LTA_max_values = get_matrix_list(filepath)[0]
                                    anomaly_max_values = np.divide(max_values, LTA_max_values)
                                    filename = write_anomaly_output(anomaly_max_values, 'FAPAR', first_date, last_date, start_year, end_year, 'maxvalues', no_value, N, region_of_interest, nameOfRegion, projection, geo_transform, -999.0)
                                    anomaly_files.append(filename)
                                    os.remove(filepath[0])
                                    
                            else:
                                period_str = str(start_date.month) + '-' + str(start_date.day) + '/' + str(end_date.month) + '-' + str(end_date.day)
                                message = 'No Long-term Average match found for "maxvalues" aggregation from the period (month-day): ' + period_str 
                                ciop.log('INFO', message)
                                
                            if len(LTA_averages) > 0:
                                LTA_averages = LTA_averages[0]
                                start_year = re.findall('\d{4}', os.path.basename(LTA_averages))[0]
                                end_year = re.findall('\d{4}', os.path.basename(LTA_averages))[1]
                                print(start_year)
                                print(end_year)
                                filepath = 'tmp_data/' + os.path.basename(LTA_averages)
                                status = get_product(LTA_averages, filepath)
                                if status == 200:
                                    filepath = [filepath]
                                    LTA_averages = get_matrix_list(filepath)[0]
                                    anomaly_averages = np.divide(averages, LTA_averages)
                                    filename = write_anomaly_output(anomaly_averages, 'FAPAR', first_date, last_date, start_year, end_year, 'averages', no_value, N, region_of_interest, nameOfRegion, projection, geo_transform, -999.0)
                                    anomaly_files.append(filename)
                                    os.remove(filepath[0])
                            
                            else:
                                period_str = str(start_date.month) + '-' + str(start_date.day) + '/' + str(end_date.month) + '-' + str(end_date.day)
                                message = 'No Long-term Average match found for "averages" aggregation from the period (month-day): ' + period_str 
                                ciop.log('INFO', message)
                            
                        
                            filenames = write_outputs('FAPAR', nameOfRegion, first_date, last_date, averages, max_values, no_value, 'GTiff', N, projection, geo_transform, -999.0)
                            filenames = filenames + anomaly_files
                            for output_name in filenames:
                                write_properties_file(interval_gpd, output_name)
                            for tmp_file in file_list:
                                os.remove(tmp_file)
                        else:
                            print('ERROR: No fAPAR product files found.')
                    
                else:
                    for month_idx in range(months_of_products[0], months_of_products[-1], n_months):
                        print(month_idx)
                        start_date = products_data[(products_data['startdate'].dt.month == month_idx)].iloc[0]['startdate']
                        end_date = products_data[(products_data['startdate'].dt.month == month_idx + n_months-1)].iloc[-1]['startdate']
                        interval_gpd = products_data[(products_data['startdate'] >= start_date) & (products_data['startdate'] <= end_date)]
                        interval_products = interval_gpd['enclosure'].tolist()
                        
                        file_list = []
                        for enclosure in interval_products:
                            filepath = product_stage_in(enclosure, 'tmp_data')
                            file_list.append(filepath)
                            print(file_list)
                        if file_list:
                            first_date = start_date.strftime('%Y-%m-%d')
                            last_date = end_date.strftime('%Y-%m-%d')
                            print(start_date)
                            print(end_date)
                            max_values, averages, no_value, geo_transform, projection = calc_aggregations(file_list, N, region_of_interest, 'FAPAR')
                        
                            LTA_max_values = get_lta_from_dataframe(lta_data, 'FAPAR', 'maxvalues', nameOfRegion, start_date.day, end_date.day, start_date.month, end_date.month, region_of_interest)
                            LTA_averages = get_lta_from_dataframe(lta_data, 'FAPAR', 'averages', nameOfRegion, start_date.day, end_date.day, start_date.month, end_date.month, region_of_interest)
                            print(LTA_max_values)
                            print(LTA_averages)
                            
                            anomaly_files = []
                            print('Calculating anomalies...')
                            if len(LTA_max_values) > 0:
                                LTA_max_values = LTA_max_values[0]
                                start_year = re.findall('\d{4}', os.path.basename(LTA_max_values))[0]
                                end_year = re.findall('\d{4}', os.path.basename(LTA_max_values))[1]
                                print(start_year)
                                print(end_year)
                                filepath = 'tmp_data/' + os.path.basename(LTA_max_values)
                                status = get_product(LTA_max_values, filepath)
                                if status == 200:
                                    filepath = [filepath]
                                    LTA_max_values = get_matrix_list(filepath)[0]
                                    anomaly_max_values = np.divide(max_values, LTA_max_values)
                                    filename = write_anomaly_output(anomaly_max_values, 'FAPAR', first_date, last_date, start_year, end_year, 'maxvalues', no_value, N, region_of_interest, nameOfRegion, projection, geo_transform, -999.0)
                                    anomaly_files.append(filename)
                                    os.remove(filepath[0])
                                    
                            else:
                                period_str = str(start_date.month) + '-' + str(start_date.day) + '/' + str(end_date.month) + '-' + str(end_date.day)
                                message = 'No Long-term Average match found for "maxvalues" aggregation from the period (month-day): ' + period_str 
                                ciop.log('INFO', message)
                                
                            
                            if len(LTA_averages) > 0:
                                LTA_averages = LTA_averages[0]
                                start_year = re.findall('\d{4}', os.path.basename(LTA_averages))[0]
                                end_year = re.findall('\d{4}', os.path.basename(LTA_averages))[1]
                                print(start_year)
                                print(end_year)
                                filepath = 'tmp_data/' + os.path.basename(LTA_averages)
                                status = get_product(LTA_averages, filepath)
                                if status == 200:
                                    filepath = [filepath]
                                    LTA_averages = get_matrix_list(filepath)[0]
                                    anomaly_averages = np.divide(averages, LTA_averages)
                                    filename = write_anomaly_output(anomaly_averages, 'FAPAR', first_date, last_date, start_year, end_year, 'averages', no_value, N, region_of_interest, nameOfRegion, projection, geo_transform, -999.0)
                                    anomaly_files.append(filename)
                                    os.remove(filepath[0])
                            
                            else:
                                period_str = str(start_date.month) + '-' + str(start_date.day) + '/' + str(end_date.month) + '-' + str(end_date.day)
                                message = 'No Long-term Average match found for "averages" aggregation from the period (month-day): ' + period_str 
                                ciop.log('INFO', message)
                        
                            filenames = write_outputs('FAPAR', nameOfRegion, first_date, last_date, averages, max_values, no_value, 'GTiff', N, projection, geo_transform, -999.0)
                            filenames = filenames + anomaly_files
                            for output_name in filenames:
                                write_properties_file(interval_gpd, output_name)
                            for tmp_file in file_list:
                                os.remove(tmp_file)
                        else:
                            print('ERROR: No fAPAR product files found.')   

[True, True, False, False, False, False, False, False, False]
[2017]
[1 2]
1
['/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_LAI-RT6_201701100000_GLOBE_PROBAV_V2.0.2.nc']
['/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_LAI-RT6_201701100000_GLOBE_PROBAV_V2.0.2.nc', '/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_LAI-RT6_201701200000_GLOBE_PROBAV_V2.0.2.nc']
['/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_LAI-RT6_201701100000_GLOBE_PROBAV_V2.0.2.nc', '/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_LAI-RT6_201701200000_GLOBE_PROBAV_V2.0.2.nc', '/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_LAI-RT6_201701310000_GLOBE_PROBAV_V2.0.2.nc']
2017-01-10
2017-01-31
<type 'list'>
c_gls_LAI-RT6_201701100000_GLOBE_PROBAV_V2.0.2.nc
255.0
c_gls_LAI-RT6_201701200000_GLOBE_PROBAV_V2.0.2.nc
255.0
c_gls_LAI-RT6_20

/opt/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:72: RuntimeWarning: invalid value encountered in divide


2015
2017


/opt/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:92: RuntimeWarning: invalid value encountered in divide


2
['/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_LAI-RT6_201702100000_GLOBE_PROBAV_V2.0.2.nc']
['/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_LAI-RT6_201702100000_GLOBE_PROBAV_V2.0.2.nc', '/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_LAI-RT6_201702200000_GLOBE_PROBAV_V2.0.2.nc']
['/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_LAI-RT6_201702100000_GLOBE_PROBAV_V2.0.2.nc', '/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_LAI-RT6_201702200000_GLOBE_PROBAV_V2.0.2.nc', '/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_LAI-RT6_201702280000_GLOBE_PROBAV_V2.0.2.nc']
2017-02-10
2017-02-28
<type 'list'>
c_gls_LAI-RT6_201702100000_GLOBE_PROBAV_V2.0.2.nc
255.0
c_gls_LAI-RT6_201702200000_GLOBE_PROBAV_V2.0.2.nc
255.0
c_gls_LAI-RT6_201702280000_GLOBE_PROBAV_V2.0.2.nc
255.0
[]
[]
Calculating anomalies...


...

/opt/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:236: RuntimeWarning: invalid value encountered in divide


2015
2017


/opt/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:257: RuntimeWarning: invalid value encountered in divide


2
['/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_FAPAR-RT6_201702100000_GLOBE_PROBAV_V2.0.2.nc']
['/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_FAPAR-RT6_201702100000_GLOBE_PROBAV_V2.0.2.nc', '/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_FAPAR-RT6_201702200000_GLOBE_PROBAV_V2.0.2.nc']
['/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_FAPAR-RT6_201702100000_GLOBE_PROBAV_V2.0.2.nc', '/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_FAPAR-RT6_201702200000_GLOBE_PROBAV_V2.0.2.nc', '/workspace/wfp-01-03-01/src/main/app-resources/notebook/libexec/tmp_data/c_gls_FAPAR-RT6_201702280000_GLOBE_PROBAV_V2.0.2.nc']
2017-02-10 00:00:00
2017-02-28 00:00:00
<type 'list'>
c_gls_FAPAR-RT6_201702100000_GLOBE_PROBAV_V2.0.2.nc
255.0
c_gls_FAPAR-RT6_201702200000_GLOBE_PROBAV_V2.0.2.nc
255.0
c_gls_FAPAR-RT6_201702280000_GLOBE_PROBAV_V2.0.2.nc
255.0

reporter:status:2019-03-29T18:01:54.018399 [INFO   ] [user process] No Long-term Average match found for "maxvalues" aggregation from the period (month-day): 2-10/2-28
2019-03-29T18:01:54.018399 [INFO   ] [user process] No Long-term Average match found for "maxvalues" aggregation from the period (month-day): 2-10/2-28
reporter:status:2019-03-29T18:01:54.018974 [INFO   ] [user process] No Long-term Average match found for "averages" aggregation from the period (month-day): 2-10/2-28
2019-03-29T18:01:54.018974 [INFO   ] [user process] No Long-term Average match found for "averages" aggregation from the period (month-day): 2-10/2-28


In [29]:
'''from matplotlib import pyplot
%matplotlib inline


fig0 = pyplot.figure()
ax0 = fig0.add_subplot(111)
cax0 = ax0.matshow(max_values)
fig0.colorbar(cax0)

fig1 = pyplot.figure()
ax1 = fig1.add_subplot(111)
cax1 = ax1.matshow(averages)
fig1.colorbar(cax1)


fig = pyplot.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(no_value)
fig.colorbar(cax)

pyplot.show()
'''

'from matplotlib import pyplot\n%matplotlib inline\n\n\nfig0 = pyplot.figure()\nax0 = fig0.add_subplot(111)\ncax0 = ax0.matshow(max_values)\nfig0.colorbar(cax0)\n\nfig1 = pyplot.figure()\nax1 = fig1.add_subplot(111)\ncax1 = ax1.matshow(averages)\nfig1.colorbar(cax1)\n\n\nfig = pyplot.figure()\nax = fig.add_subplot(111)\ncax = ax.matshow(no_value)\nfig.colorbar(cax)\n\npyplot.show()\n'

### <a name="license">License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.